In [1]:
import requests
from scrapy.selector import Selector as scp
import sys
from unicodedata import normalize

class Search(object):
	"""docstring for Search"""
	def __init__(self, palavra):
		super(Search, self).__init__()
		self.word = palavra.split(" ")

	def synonyms(self, verbose=False):
		param = "-".join(self.word)
		param = normalize('NFKD', param).encode('ASCII','ignore').decode('ASCII')
		if verbose == True:
			print("Carregando sinônimos para '{}'...".format(param))
			try:
				r = requests.get('https://www.sinonimos.com.br/{}/'.format(param))
				print(r)
				if r.status_code == 200:
					conteudo = r.content.decode('iso8859-1')
					#palavra = scp(text=conteudo).xpath('//h1[@class="h-palavra"]/text()').extract_first()[12:]
					sinonimos = scp(text=conteudo).xpath('//a[@class="sinonimo"]/text()').extract()
					#tam_sinonimos = len(sinonimos)

					#print("{} resultados encontrados:".format(tam_sinonimos))
					
					#arq = open('.historico', 'a')
					#arq.write("{} resultados encontrados para '{}':\n".format(tam_sinonimos, palavra))
					#arq.write(str(sinonimos))
					#arq.write("\n\n")
					#arq.close()
					#print("Pronto!")
					return sinonimos
				else:
					return "Nenhum resultado encontrado."
			except Exception as e:
				return "Impossivel conectar a internet. :/. Tipo de erro: {}".format(e)
		else:
			try:
				r = requests.get('https://www.sinonimos.com.br/{}/'.format(param))
				if r.status_code == 200:
					conteudo = r.content.decode('iso8859-1')
					#palavra = scp(text=conteudo).xpath('//h1[@class="h-palavra"]/text()').extract_first()[12:]
					sinonimos = scp(text=conteudo).xpath('//a[@class="sinonimo"]/text()').extract()
					#tam_sinonimos = len(sinonimos)
					#arq = open('.historico', 'a')
					#arq.write("{} resultados encontrados para '{}':\n".format(tam_sinonimos, param))
					#arq.write(str(sinonimos))
					#arq.write("\n\n")
					#arq.close()
					return sinonimos
				else:
					return []
			except Exception as e:
				return e

def about():
	return "Bem-vindo ao pysinonimos!\nPara usar execute: python pysinonimos.py 'palavra'"

def historic(delete=False):
	try:
		if delete == True:
			arq = open('.historico', 'w')
			arq.write("")
			arq.close()
			print("Histórico apagado.")
		else:
			arq = open('.historico', 'r')
			return arq.read()
			arq.close()
	except Exception as e:
		return "Arquivo ainda não existe."

In [21]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
df = pd.read_csv('tratados-manual-corrigido.csv')
print("numero de registros:", len(df. index))
#df = df.loc[df["Comparador"] == 1,['PROBLEMA/RESOLUÇÃO', 'Pergunta', 'Resposta']]
df.head()

numero de registros: 920


,PROBLEMA/RESOLUÇÃO,Pergunta,Resposta
0,como consegui informe de rendimentos? en viar email para caj e tramitamos para a dag.,como consegui informe de rendimentos,en vir email para caj e tramitamos para a dag .
1,"Caro(a), servidor(a).\n\nSou Francisnilton Moura, advogado, inscrito na OAB/RN 8851, estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990, requerido Cortez Pereira de Araújo Júnior e não consigo. Assim solicita-se o envio da citada decisão por email. Já falei com vocês o ano passado.\nAtenciosamente. Mossoró/RN, 09 de junho de 2021. Francisnilton Moura OAB/RN 8851. \n\nR: Ao consultar esse processo verificamos que o mesmo já foi transitado e julgado e devolvido para a Prefeitura Municipal de Serra do Mel, para maiores informações, sugerimos entrar em contato com a Diretoria de Expediente através do contato telefônico 3642.7371 ou por e-mail: de.tcern@gmail.com.\n","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , requerido Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . Atenciosamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Ao consultar esse processo verificamos que o mesmo já foi transitado e julgado e devolvido para a Prefeitura Municipal de Serra do Mel , para maiores informações , sugerimos entrar em contato com a Diretoria de Expediente através do contato telefônico 3642.7371 ou por e-mail : de.tcern @ gmail.com ."
2,"Não aparece tela de erro, no acesso pela área restrita apenas aparece novamente a janelinha para inserir o usuário e senha e no portal do gestor aparece em vermelho que o usuário não foi encontrado (anexos).\n\nMeu CPF é 09875668494 - Julia de Renor Oliveira Campos\n\nR: Ao realizar consulta em nossa base de dados verificamos que o seu cadastro é de usuário fiscalizador, o qual deve ser acessado através do link da Área Restrita disponibilizada no site do TCE, conforme print.\nSegue senha que foi restaurada para utilização do seu acesso: Tce.2021#\n","Não aparece tela de erro , no acesso pela área restrita apenas aparece novamente a janelinha para inserir o usuário e senha e no portal do gestor aparece em vermelho que o usuário não foi encontrado ( anexos ) . Meu CPF é 09875668494 - Julia de menor Oliveira Campos","Ao realizar consulta em nossa base de dados verificamos que o seu cadastro é de usuário fiscalizador , o qual deve ser acessado através do link da Área Restrita disponibilizada no site do TCE , conforme print . Segue senha que foi restaurada para utilização do seu acesso : Tce.2021 #"
3,PREVIDÊNCIA DE CEARÁ MIRIM- Processo de aposentadoria foi encaminhado pelo Portal ETCE e foi negado o recebimento.\n\nSolicitado entrar em contato com a DE para saber o motivo da negação do envio.,PREVIDÊNCIA DE CEARÁ minimo Processo de aposentadoria foi encaminhado pelo Portal ETCE e foi negado o recebimento .,Solicitado entrar em contato com a DE para saber o motivo da negação do envio .
4,"Memorando 311/2021 - Gostaria do passo a passo para encerrar o vinculo de servidor ja exonerado no Sispatri\n\n\nR: Com cordiais saudações, informamos que para realizar qualquer alteração de dados de usuário cadastrado no SISPATRI, é necessário acessar o sistema através do link: https://sispatri.tce.rn.gov.br/PaginasPublicas/login.aspx, e em seguida na opção Vínculo inserir no campo data da saída, a data da exoneração do servidor, para que assim o sistema compreenda sua saída do cargo. Lembrando que seus rendimentos de saída deverão ser informados e encaminhados no prazo de 30 dias, para que assim sua situação continue regular com esta Corte de Contas. Qualquer dúvida que venha a surgir entrar em contato diretamente com a Central de Atendimento ao Jurisdicionado (CAJ) deste Tribunal por meio do telefone 3642-7275 ou e-mail caj@tce.rn.

In [3]:
#inicialização corretor ortográfico
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from unidecode import unidecode
import random
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
language = "portuguese"
stop_words = set(stopwords.words(language))
from spellchecker import SpellChecker
spell = SpellChecker(language='pt')
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
from autocorrect import Speller
speller = Speller(lang='pt')

import nltk
nltk.download('mac_morpho')
nltk.corpus.mac_morpho.words()
universetag = nltk.corpus.mac_morpho.tagged_words()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ormaz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ormaz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing B

In [4]:
nomes_proprios = []

for x in universetag:
    if x[1] == "NPROP":
        nomes_proprios.append(x[0].lower())
        
nomes_proprios.append("caj")
nomes_proprios.append("dag")
nomes_proprios.append("tce")
nomes_proprios.append("dap")
nomes_proprios.append("e-sic")
nomes_proprios.append("cpf")
nomes_proprios.append("saae")
nomes_proprios.append("e-tce")
nomes_proprios.append("etce")
nomes_proprios.append("oab")

In [29]:
#cria colunas teste
#df2 = df
#df2.insert(3, column = "Pergunta Corrigida", value = df["Pergunta"])
#df2.insert(4, column = "Resposta Corrigida", value = df["Resposta"])
#df2.head()

In [5]:
import re
def findWholeWord(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

dt = pd.read_json("AcordaosRelatorios.json", lines=True)
listCorpusJuridico = dt['texto'].apply(lambda x: x.lower())

arrayCorpusJuridico = []
arrayCorpusJuridico.append("tramitamos")

def inCorpusJuridico(text):
    for frase in listCorpusJuridico:
        if findWholeWord(text)(frase):
            return True
    if text in arrayCorpusJuridico:
        return True
    return False

In [6]:
def corretorOrtografico(text):
    print("entrada: " + str(text))
    tokens = tokenizer.tokenize(text, return_tensors='pt')
    word_tokens = word_tokenize(text)
    saida = ""
    #print(word_tokens)
    for w in word_tokens:
        if w not in stop_words and w.lower() not in nomes_proprios and w not in tokens and not inCorpusJuridico(w):
            try:
                #print(w)
                saida = saida + " " + spell.correction(w)
            except:
                saida = saida + " " + w
            #print(saida)
        else:
            saida = saida + " " + w   
    print("saida: " + str(saida))
    return saida

In [14]:
#teste de palavras
w = "gabinetess"
word_tokens = word_tokenize(w)
print(word_tokens)
tokens = tokenizer.tokenize(w, return_tensors='pt')
print(w not in stop_words)
print(w.lower() not in nomes_proprios)
print(w not in tokens)
print(not inCorpusJuridico(w))
spell.correction(w)

['gabinetess']
True
True
True
True


'gabinetes'

In [17]:
from itertools import islice
#corretor ortográfico
runCorretorOrtografico = 0
df2 = df
for index, row in df2.iterrows():
#for index, row in islice(df2.iterrows(), 582, None):
    #corretorOrtografico(row['Pergunta'])
    #corretorOrtografico(row['Resposta'])
    df2.at[index,'Pergunta'] = corretorOrtografico(row['Pergunta'])
    df2.at[index,'Resposta'] = corretorOrtografico(row['Resposta'])
    
    runCorretorOrtografico = runCorretorOrtografico + 1
    porcentagem = (runCorretorOrtografico/len(df.index))*100
    print("{:.2f}%".format(porcentagem))

entrada: como consegui informe de rendimentos
saida:  como consegui informe de rendimentos
entrada: en viar email para caj e tramitamos para a dag.
saida:  en vir email para caj e tramitamos para a dag .
14.29%
entrada: 7513/2016 - Duida sore processo aposentadoria
saida:  7513/2016 - cuida sore processo aposentadoria
entrada: solicitado ligar para a DAP
saida:  solicitado ligar para a DAP
28.57%
entrada: qual a ferramenta para solicitação infomração
saida:  qual a ferramenta para solicitação informação
entrada: E-sic
saida:  E-sic
42.86%
entrada: Natalia –pm touros – e-tce – fez o cadastro e ainda não recebeu a senha ?
saida:  Natalia pm touros – e-tce – fez o cadastro e ainda não recebeu a senha ?
entrada:     foi informado a mesma que demora 1 dia útil e q ela deverá receber até o final do dia.
saida:  foi informado a mesma que demora 1 dia útil e q ela deverá receber até o final do dia .
57.14%
entrada: envie o doc para a dag e peça refri e pizsas renato
saida:  envie o doc para a 

In [13]:
printer = df2[["PROBLEMA/RESOLUÇÃO", "Pergunta", "Resposta"]]
printer.to_csv('tratados-manual-corrigido-lite.csv', index=False)
df2

,PROBLEMA/RESOLUÇÃO,Pergunta,Resposta
0,como consegui informe de rendimentos? en viar email para caj e tramitamos para a dag.,como consegui informe de rendimentos,en vir email para caj e tramitamos para a dag .
1,7513/2016 - Duida sore processo aposentadoria\n\nsolicitado ligar para a DAP,7513/2016 - cuida sore processo aposentadoria,solicitado ligar para a DAP
2,qual a ferramenta para solicitação infomração? E-sic,qual a ferramenta para solicitação informação,E-sic
3,Natalia –pm touros – e-tce – fez o cadastro e ainda não recebeu a senha ? foi informado a mesma que demora 1 dia útil e q ela deverá receber até o final do dia.,Natalia pm touros – e-tce – fez o cadastro e ainda não recebeu a senha ?,foi informado a mesma que demora 1 dia útil e q ela deverá receber até o final do dia .
4,envie o doc para a dag e peça refri e pizsas renato,envie o doc para a dag e peça referi e pizzas renato,envie o doc para a dag e peça referi e pizzas renato
5,Tinha uma pizsas no meio do caminho.,Tinha uma pizzas no meio do caminho .,Tinha uma pizzas no meio do caminho .
6,Telefone do gab do cons dr Renato,Telefone do gab do cons dr Renato,Telefone do gab do cons dr Renato


In [23]:
#gerador de novos exemplos
#df2 = df
nPerguntas = 5
for count in range(2, nPerguntas + 2):
    df2.insert(count + 1, column = "Pergunta" + str(count), value = df["Pergunta"])
df2.head()

,PROBLEMA/RESOLUÇÃO,Pergunta,Resposta,Pergunta2,Pergunta3,Pergunta4,Pergunta5,Pergunta6
0,como consegui informe de rendimentos? en viar email para caj e tramitamos para a dag.,como consegui informe de rendimentos,en vir email para caj e tramitamos para a dag .,como consegui informe de rendimentos,como consegui informe de rendimentos,como consegui informe de rendimentos,como consegui informe de rendimentos,como consegui informe de rendimentos
1,"Caro(a), servidor(a).\n\nSou Francisnilton Moura, advogado, inscrito na OAB/RN 8851, estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990, requerido Cortez Pereira de Araújo Júnior e não consigo. Assim solicita-se o envio da citada decisão por email. Já falei com vocês o ano passado.\nAtenciosamente. Mossoró/RN, 09 de junho de 2021. Francisnilton Moura OAB/RN 8851. \n\nR: Ao consultar esse processo verificamos que o mesmo já foi transitado e julgado e devolvido para a Prefeitura Municipal de Serra do Mel, para maiores informações, sugerimos entrar em contato com a Diretoria de Expediente através do contato telefônico 3642.7371 ou por e-mail: de.tcern@gmail.com.\n","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , requerido Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . Atenciosamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Ao consultar esse processo verificamos que o mesmo já foi transitado e julgado e devolvido para a Prefeitura Municipal de Serra do Mel , para maiores informações , sugerimos entrar em contato com a Diretoria de Expediente através do contato telefônico 3642.7371 ou por e-mail : de.tcern @ gmail.com .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , requerido Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . Atenciosamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , requerido Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . Atenciosamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , requerido Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . Atenciosamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , requerido Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . Atenciosamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , requerido Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . Atenciosamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 ."
2,"Não aparece tela de erro, no ac

In [24]:
maximoPalavras = 2
for index, row in df2.iterrows():
    sentence = row['Pergunta']
    if str(sentence).strip() != "":
        print("iteração numero " + str(index))
        print("pergunta selecionada: " + str(sentence))
        
        text = re.sub(r'[.,"\'-?:!;–]', '', str(sentence))
        text = re.sub(r'\S*@\S*\s?', '', text)
        word_tokens = word_tokenize(text)
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        filtered_sentence.sort(key=len)
        
        numWords = min(6, len(filtered_sentence))
        maxWords = min(maximoPalavras, numWords)
        counterPalavras = 0
        most_valuable_words = []
        
        print("maximo de palavras a utilizar: " + str(numWords))
        print("numero de palavras a utilizar: " + str(maxWords))
        
        #preenche vetor de palavras a serem modificadas
        while(counterPalavras < numWords):
            most_valuable_words.append(filtered_sentence[len(filtered_sentence)-1-counterPalavras])
            counterPalavras = counterPalavras + 1
            
        print(most_valuable_words)
        counterPerguntas = 0
        counterPalavras = 0
        
        while(counterPalavras < maxWords):
            
            selected_word_orig = most_valuable_words[counterPalavras]
            selected_word = unidecode(most_valuable_words[counterPalavras])
            word = Search(selected_word)
            sinonimos = word.synonyms()
            
            print(selected_word_orig)
            print(sinonimos)
            
            if(len(sinonimos)):
            
                while(counterPerguntas < nPerguntas):

                    nova_pergunta = df2.at[index,'Pergunta' + str(counterPerguntas+2)].replace(selected_word_orig, sinonimos[min(counterPerguntas, len(sinonimos)-1)])
                    print(nova_pergunta)
                    df2.at[index,'Pergunta' + str(counterPerguntas+2)] = nova_pergunta
                    counterPerguntas = counterPerguntas + 1
            
            elif(maxWords < numWords):
                
                maxWords = maxWords + 1
            
            counterPerguntas = 0
            counterPalavras = counterPalavras + 1    

df2.to_csv('saida.csv', index=False)
df2.head()

iteração numero 0
pergunta selecionada:  como consegui informe de rendimentos
maximo de palavras a utilizar: 3
numero de palavras a utilizar: 2
['rendimentos', 'consegui', 'informe']
rendimentos
['possibilidade', 'capacidade', 'haveres', 'posses', 'bens', 'possibilidades', 'capacidades', 'riqueza', 'acervo', 'fortuna', 'capital', 'espólio', 'dinheiro', 'tesouro', 'fazenda', 'herança', 'pecúlio', 'cabedal', 'chorume', 'dote', 'economias', 'meios', 'recursos', 'investimento', 'patrimônio', 'pertences', 'vinténs', 'propriedades', 'divícia', 'fundos', 'faculdades', 'coisas', 'valores', 'benefícios', 'juros', 'dividendos', 'ganhos', 'lucros', 'subsídios', 'honorários', 'proventos', 'benesses', 'proveitos', 'riquezas', 'interesses', 'prêmio', 'resultados', 'produtos', 'férias', 'vantagens', 'posse']
 como consegui informe de possibilidade
 como consegui informe de capacidade
 como consegui informe de haveres
 como consegui informe de posses
 como consegui informe de bens
consegui
['pude', 't

janelinha
[]
novamente
['de novo', 'outra vez', 'mais uma vez', 'repetidamente', 'reiteradamente', 'recentemente', 'há pouco']
 Não aparece tela de erro , no acesso pela área restrita apenas aparece de novo a janelinha para inserir o usuário e senha e no portal do gestor aparece em vermelho que o usuário não foi achado ( anexos ) . Meu CPF é 09875668494 - Julia de menor Oliveira Campos
 Não aparece tela de erro , no acesso pela área restrita apenas aparece outra vez a janelinha para inserir o usuário e senha e no portal do gestor aparece em vermelho que o usuário não foi descoberto ( anexos ) . Meu CPF é 09875668494 - Julia de menor Oliveira Campos
 Não aparece tela de erro , no acesso pela área restrita apenas aparece mais uma vez a janelinha para inserir o usuário e senha e no portal do gestor aparece em vermelho que o usuário não foi visto ( anexos ) . Meu CPF é 09875668494 - Julia de menor Oliveira Campos
 Não aparece tela de erro , no acesso pela área restrita apenas aparece repet

tentando
['procurando', 'buscando', 'intentando', 'pretendendo', 'realizando', 'investigando', 'pesquisando']
 Xavier - e-sic - - 27764435000169 , usuário está procurando acesso e não atinge .
 Xavier - e-sic - - 27764435000169 , usuário está buscando acesso e não chega .
 Xavier - e-sic - - 27764435000169 , usuário está intentando acesso e não conquista .
 Xavier - e-sic - - 27764435000169 , usuário está pretendendo acesso e não realiza .
 Xavier - e-sic - - 27764435000169 , usuário está realizando acesso e não cumpre .
iteração numero 8
pergunta selecionada:  Usuário gostaria de confirmar email do e-sic . Cláudio Gomes de Medeiros Filho CPF 011 978 904 32 .
maximo de palavras a utilizar: 6
numero de palavras a utilizar: 2
['confirmar', 'Medeiros', 'gostaria', 'Cláudio', 'Usuário', 'Filho']
confirmar
['comprovar', 'provar', 'atestar', 'ratificar', 'corroborar', 'asseverar', 'assegurar', 'afirmar', 'certificar', 'demonstrar', 'conferir', 'constatar', 'verificar', 'solidar', 'solidifica

solicitar
['pedir', 'implorar', 'suplicar', 'instar', 'rogar', 'exigir', 'reclamar', 'pedinchar', 'clamar', 'convocar', 'obsecrar', 'requerer', 'requisitar', 'demandar', 'impetrar', 'pleitear', 'procurar', 'buscar', 'diligenciar', 'granjear', 'tentar', 'intentar', 'agenciar', 'provocar', 'incitar', 'instigar', 'impelir', 'estimular', 'desafiar', 'convidar', 'encorajar', 'levar', 'chamar', 'induzir', 'arrastar', 'conduzir', 'preocupar-se', 'inquietar-se', 'desassossegar-se', 'afligir-se', 'perturbar-se', 'atrair']
 com pedir conhecimento ao tce rn
 com implorar referência ao tce rn
 com suplicar noção ao tce rn
 com instar ideia ao tce rn
 com rogar base ao tce rn
iteração numero 13
pergunta selecionada:  Gostaria de solicitar os Dados dos Servidores ( efetivos e contratos temporário ) e suas Remunerações no primeiro semestre de 2021 .
maximo de palavras a utilizar: 6
numero de palavras a utilizar: 2
['Remunerações', 'temporário', 'Servidores', 'contratos', 'solicitar', 'semestre']
Remu

obter
['conseguir', 'alcançar', 'atingir', 'adquirir', 'lograr', 'merecer', 'abiscoitar', 'arranjar', 'angariar', 'arrecadar', 'coletar', 'colher', 'captar', 'arrumar', 'ganhar', 'conquistar', 'granjear', 'atrair', 'cativar', 'receber', 'ter', 'tirar', 'levar', 'apanhar', 'auferir', 'lucrar']
 como conseguir conhecimentos do tce
 como alcançar referências do tce
 como atingir noções do tce
 como adquirir ideias do tce
 como lograr pesquisas do tce
iteração numero 18
pergunta selecionada:  Sou Luiz Batista ex vereador de Maxaranguape-Rn , veio por meio deste e-mail para solicitar a última folha de pagamento do município de Rio do fogo -Rn com o SAAE . Desde de já agradeço
maximo de palavras a utilizar: 6
numero de palavras a utilizar: 2
['MaxaranguapeRn', 'município', 'pagamento', 'solicitar', 'agradeço', 'vereador']
MaxaranguapeRn
[]
município
['cidade', 'localidade', 'urbanização', 'povoação', 'burgo', 'urbe', 'povoamento', 'prefeitura', 'câmara', 'câmara municipal', 'administração', 

...

processamento
['tratamento', 'computação', 'processo', 'seguimento', 'continuação', 'encadeamento', 'sequência', 'série', 'sucessão', 'curso', 'preparação', 'preparo', 'preparamento', 'preparativo', 'feitura', 'elaboração', 'organização', 'manipulação']
 28/05 PM Nísia floresta – mensagem do tratamento apresentou erro e usuário não sabe o q fazer .
 28/05 PM Nísia floresta – mensagem do computação apresentou erro e usuário não sabe o q fazer .
 28/05 PM Nísia floresta – mensagem do processo apresentou erro e usuário não sabe o q fazer .
 28/05 PM Nísia floresta – mensagem do seguimento apresentou erro e usuário não sabe o q fazer .
 28/05 PM Nísia floresta – mensagem do continuação apresentou erro e usuário não sabe o q fazer .
apresentou
['mostrou', 'passou', 'demonstrou', 'relatou', 'teve', 'trouxe', 'apareceu', 'foi', 'surgiu', 'deu', 'propôs']
 28/05 PM Nísia floresta – mensagem do tratamento mostrou erro e usuário não sabe o q fazer .
 28/05 PM Nísia floresta – mensagem do computa

problema
['contrariedade', 'adversidade', 'contratempo', 'dificuldade', 'sufoco', 'obstáculo', 'revés', 'objeção', 'transtorno', 'complicação', 'quebra-cabeça', 'pepino', 'abacaxi', 'infortúnio', 'porém', 'aborrecimento', 'inconveniente', 'embaraço', 'incômodo', 'impedimento', 'oposição', 'resistência', 'atrapalhação', 'embrulhada', 'imbróglio', 'mas', 'atribulação', 'aperto', 'galho', 'senão', 'embrulho', 'buraco', 'grilo', 'espeto', 'empecilho', 'óbice', 'vicissitude', 'distúrbio', 'disfunção', 'perturbação', 'defeito', 'imperfeição', 'falha', 'doença', 'caso', 'vício', 'dúvida', 'dilema', 'impasse', 'pergunta', 'interrogação', 'assunto', 'tema', 'questão', 'ponto', 'quesito', 'enigma', 'mistério', 'incógnita', 'segredo', 'charada']
 PM serra negra do norte – Diana – criar cadastro para enviar conhecimentos do sai – 42805070828 . Usuária informa que foi criado um usuário pra ela mas a mesma não recebeu a senha devido a contrariedade no email .
 PM serra negra do norte – Diana – criar

extremo
['afastado', 'distante', 'remoto', 'retirado', 'longe', 'longínquo', 'distanciado', 'separado', 'desviado', 'apartado', 'arredado', 'remontado', 'semoto', 'intenso', 'máximo', 'supremo', 'forte', 'acentuado', 'grande', 'violento', 'veemente', 'carregado', 'ativo', 'enérgico', 'ardente', 'grave', 'preocupante', 'sério', 'inquietante', 'crítico', 'dramático', 'delicado', 'momentoso', 'ponderoso', 'anormal', 'excepcional', 'último', 'derradeiro', 'final', 'terminal', 'terminante', 'terminativo', 'fatal', 'radical', 'drástico', 'extremista', 'extremado', 'intransigente', 'excessivo', 'exagerado', 'extremidade', 'ponta', 'fim', 'limite', 'limiar', 'raia', 'termo', 'exageros', 'excessos', 'descomedimentos', 'cuidados', 'carinhos', 'desvelo', 'zelo', 'ternura']
 Dúvida sobre acesso ao Portal do Gestor - Pm afastado
 Dúvida sobre acesso ao Portal do Gestor - Pm distante
 Dúvida sobre acesso ao Portal do Gestor - Pm remoto
 Dúvida sobre acesso ao Portal do Gestor - Pm retirado
 Dúvida s

informou
['disse', 'participou']
 cosern - usuário gostaria de alterar a senha e disse que o local atribuído a isso não deu certo .
 cosern - usuário gostaria de alterar a senha e participou que o local talhado a isso não deu certo .
 cosern - usuário gostaria de alterar a senha e participou que o local designado a isso não deu certo .
 cosern - usuário gostaria de alterar a senha e participou que o local determinado a isso não deu certo .
 cosern - usuário gostaria de alterar a senha e participou que o local fadado a isso não deu certo .
iteração numero 919
pergunta selecionada:  PM monte alegre Cadastro Gerenciador09642086441 - Ruan Pablo Medeiros
maximo de palavras a utilizar: 6
numero de palavras a utilizar: 2
['Gerenciador', 'Medeiros', 'Cadastro', 'alegre', 'Pablo', 'monte']
Gerenciador
[]
Medeiros
[]
Cadastro
['arquivo', 'ficha', 'fichário', 'registro', 'censo', 'estatística', 'recenseamento', 'catálogo', 'elenco', 'enumeração', 'guia', 'índice', 'inventário', 'lista', 'listagem

,PROBLEMA/RESOLUÇÃO,Pergunta,Resposta,Pergunta2,Pergunta3,Pergunta4,Pergunta5,Pergunta6
0,como consegui informe de rendimentos? en viar email para caj e tramitamos para a dag.,como consegui informe de rendimentos,en vir email para caj e tramitamos para a dag .,como pude informe de possibilidade,como tive informe de capacidade,como tive informe de haveres,como tive informe de posses,como tive informe de bens
1,"Caro(a), servidor(a).\n\nSou Francisnilton Moura, advogado, inscrito na OAB/RN 8851, estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990, requerido Cortez Pereira de Araújo Júnior e não consigo. Assim solicita-se o envio da citada decisão por email. Já falei com vocês o ano passado.\nAtenciosamente. Mossoró/RN, 09 de junho de 2021. Francisnilton Moura OAB/RN 8851. \n\nR: Ao consultar esse processo verificamos que o mesmo já foi transitado e julgado e devolvido para a Prefeitura Municipal de Serra do Mel, para maiores informações, sugerimos entrar em contato com a Diretoria de Expediente através do contato telefônico 3642.7371 ou por e-mail: de.tcern@gmail.com.\n","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , requerido Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . Atenciosamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Ao consultar esse processo verificamos que o mesmo já foi transitado e julgado e devolvido para a Prefeitura Municipal de Serra do Mel , para maiores informações , sugerimos entrar em contato com a Diretoria de Expediente através do contato telefônico 3642.7371 ou por e-mail : de.tcern @ gmail.com .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , solicitado Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . gentilmente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , pedido Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . delicadamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , suplicado Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . cortesmente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , implorado Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . obsequiosamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , rogado Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . polidamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 ."
2,"Não aparece tela de erro, no acesso pela área restrita apenas aparece novamente 

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
df2 = pd.read_csv('saida.csv')
df2

,PROBLEMA/RESOLUÇÃO,Pergunta,Resposta,Pergunta2,Pergunta3,Pergunta4,Pergunta5,Pergunta6
0,como consegui informe de rendimentos? en viar email para caj e tramitamos para a dag.,como consegui informe de rendimentos,en vir email para caj e tramitamos para a dag .,como pude informe de possibilidade,como tive informe de capacidade,como tive informe de haveres,como tive informe de posses,como tive informe de bens
1,"Caro(a), servidor(a).\n\nSou Francisnilton Moura, advogado, inscrito na OAB/RN 8851, estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990, requerido Cortez Pereira de Araújo Júnior e não consigo. Assim solicita-se o envio da citada decisão por email. Já falei com vocês o ano passado.\nAtenciosamente. Mossoró/RN, 09 de junho de 2021. Francisnilton Moura OAB/RN 8851. \n\nR: Ao consultar esse processo verificamos que o mesmo já foi transitado e julgado e devolvido para a Prefeitura Municipal de Serra do Mel, para maiores informações, sugerimos entrar em contato com a Diretoria de Expediente através do contato telefônico 3642.7371 ou por e-mail: de.tcern@gmail.com.\n","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , requerido Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . Atenciosamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Ao consultar esse processo verificamos que o mesmo já foi transitado e julgado e devolvido para a Prefeitura Municipal de Serra do Mel , para maiores informações , sugerimos entrar em contato com a Diretoria de Expediente através do contato telefônico 3642.7371 ou por e-mail : de.tcern @ gmail.com .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , solicitado Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . gentilmente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , pedido Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . delicadamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , suplicado Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . cortesmente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , implorado Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . obsequiosamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 .","Caro ( a ) , servidor ( a ) . Sou Francisnilton Moura , advogado , inscrito na OAB/RN 8851 , estou tentando baixar na aba consulta de processo a decisão do processo de 01136/1990 , rogado Cortez Pereira de Araújo Júnior e não consigo . Assim solicita-se o envio da citada decisão por email . Já falei com vocês o ano passado . polidamente . Mossoró/RN , 09 de junho de 2021 . Francisnilton Moura OAB/RN 8851 ."
2,"Não aparece tela de erro, no acesso pela área restrita apenas aparece novamente 

In [3]:
file = open('nlu.yml', 'w')
L = ["version: \"3.1\"\n", "nlu:\n"]
file.writelines(L)
for x in range(len(df2. index)):
    try:
        if str(df2.loc[x, 'Pergunta']).strip() != "" and str(df2.loc[x, 'Pergunta2']).strip() != "" and str(df2.loc[x, 'Pergunta3']).strip() != "" and str(df2.loc[x, 'Pergunta4']).strip() != "" and str(df2.loc[x, 'Pergunta5']).strip() != "" and str(df2.loc[x, 'Pergunta6']).strip() != "":
            d = str(df2.loc[x, 'Pergunta']).replace("\n","")
            d = d.replace(":","")
            s = "- intent: " + "int" + str(x) + "\n"
            s = s + "  examples: |\n"
            s = s + "    - " + d + "\n"
            if str(df2.loc[x, 'Pergunta2']).strip() != "":
                d = str(df2.loc[x, 'Pergunta2']).replace("\n","")
                d = d.replace(":","")
                s = s + "    - " + d + "\n"
            if str(df2.loc[x, 'Pergunta3']).strip() != "":
                d = str(df2.loc[x, 'Pergunta3']).replace("\n","")
                d = d.replace(":","")
                s = s + "    - " + d + "\n"
            if str(df2.loc[x, 'Pergunta4']).strip() != "":
                d = str(df2.loc[x, 'Pergunta4']).replace("\n","")
                d = d.replace(":","")
                s = s + "    - " + d + "\n"
            if str(df2.loc[x, 'Pergunta5']).strip() != "":
                d = str(df2.loc[x, 'Pergunta5']).replace("\n","")
                d = d.replace(":","")
                s = s + "    - " + d + "\n"
            if str(df2.loc[x, 'Pergunta6']).strip() != "":
                d = str(df2.loc[x, 'Pergunta6']).replace("\n","")
                d = d.replace(":","")
                s = s + "    - " + d + "\n"
            s = s + "\n"
            file.write(s)
    except KeyError:
        pass

file.close()

In [4]:
file = open('domain.yml', 'w')
L = ["version: \"3.1\"\n\n", "intents:\n"]
file.writelines(L)

for x in range(len(df2. index)):
    try:
        if str(df2.loc[x, 'Pergunta']).strip() != "" and str(df2.loc[x, 'Pergunta2']).strip() != "" and str(df2.loc[x, 'Pergunta3']).strip() != "" and str(df2.loc[x, 'Pergunta4']).strip() != "" and str(df2.loc[x, 'Pergunta5']).strip() != "" and str(df2.loc[x, 'Pergunta6']).strip() != "":    
            d = str(df2.loc[x, 'Resposta'])
            s = "  - int" + str(x) + "\n"
            file.write(s)
    except KeyError:
        pass

file.write("\nresponses: \n")

for x in range(len(df2. index)):
    try:
        if str(df2.loc[x, 'Pergunta']).strip() != "" and str(df2.loc[x, 'Pergunta2']).strip() != "" and str(df2.loc[x, 'Pergunta3']).strip() != "" and str(df2.loc[x, 'Pergunta4']).strip() != "" and str(df2.loc[x, 'Pergunta5']).strip() != "" and str(df2.loc[x, 'Pergunta6']).strip() != "":
            d = str(df2.loc[x, 'Resposta']).replace("\n","")
            d = d.replace(":","")
            s = "  utter_int" + str(x) + ":\n"
            s = s + "  - text: \"" + d.strip() + "\"\n\n"
            file.write(s)
    except KeyError:
        pass

file.close()

In [5]:
file = open('rules.yml', 'w')
L = ["version: \"3.1\"\n\n", "rules:\n\n"]
file.writelines(L)

for x in range(len(df2. index)):
    try:
        if str(df2.loc[x, 'Pergunta']).strip() != "" and str(df2.loc[x, 'Pergunta2']).strip() != "" and str(df2.loc[x, 'Pergunta3']).strip() != "" and str(df2.loc[x, 'Pergunta4']).strip() != "" and str(df2.loc[x, 'Pergunta5']).strip() != "" and str(df2.loc[x, 'Pergunta6']).strip() != "":
            d = str(df2.loc[x, 'Resposta'])
            s = "- rule: int" + str(x) + "\n" + "  steps:\n"
            s = s + "  - intent: int" + str(x) + "\n"
            s = s + "  - action: utter_int" + str(x) + "\n\n"
            file.write(s)
    except KeyError:
        pass

file.close()